In [1]:
import gc
from glob import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import time

import category_encoders as ce
import catboost
from sklearn import metrics

import sys
sys.path.append("../utils")
from metrics import compute_recall_at4, compute_normalized_gini, compute_amex_metric
#from messaging import send_message

pd.set_option('display.max_columns', None)

In [2]:
# metrics in catboost format
class AmexMetric:
    
    def is_max_optimal(self):
        return True

    def evaluate(self, approxes, target, weight):    
        assert len(approxes) == 1
        assert len(target) == len(approxes[0])
        scores = approxes[0]
        target = target
        return compute_amex_metric(target, scores), 1.

    def get_final_error(self, error, weight):
        return error

class RecallAt4:
    
    def is_max_optimal(self):
        return True

    def evaluate(self, approxes, target, weight):
        assert len(approxes) == 1
        assert len(target) == len(approxes[0])
        scores = np.array(approxes[0])
        target = np.array(target)                                   
        return compute_recall_at4(target, scores), 1.

    def get_final_error(self, error, weight):
        return error

class NormGini:
    
    def is_max_optimal(self):
        return True

    def evaluate(self, approxes, target, weight):
        assert len(approxes) == 1
        assert len(target) == len(approxes[0])
        scores = np.array(approxes[0])
        target = np.array(target)                   
        return compute_normalized_gini(target, scores), 1.

    def get_final_error(self, error, weight):
        return error

In [3]:
# CONFIG PARAMS
N_REPEATS = 3

In [4]:
OOF_PATH = Path("../data/oof/catboost-dsv06")
SUB_PATH = Path("../data/subs/catboost-dsv06")
ART_PATH = Path("../artifacts/catboost-dsv06")

if not OOF_PATH.exists():
    OOF_PATH.mkdir(parents=True, exist_ok=True)
if not SUB_PATH.exists():
    SUB_PATH.mkdir(parents=True, exist_ok=True)
if not ART_PATH.exists():
    ART_PATH.mkdir(parents=True, exist_ok=True)

***
## load and prepare data

In [5]:
!ls ../data/processed/dsv06

test.parquet  train.parquet


In [6]:
train = pd.read_parquet("../data/processed/dsv06/train.parquet")
train_labels = pd.read_csv("../data/raw/train_labels.csv", index_col="customer_ID")

In [7]:
input_feats = train.columns.tolist()
len(input_feats)

1308

In [8]:
train = pd.merge(train, train_labels, how="inner", left_index=True, right_index=True)
del train_labels
gc.collect()

0

***
## model training

train with repeated cross validation

In [9]:
model_params = {
    'eval_metric':AmexMetric(),
    'learning_rate': 0.05,
    'nan_mode':'Min',
    'random_seed': 2112,
    'auto_class_weights': None,
    'bootstrap_type': 'Bernoulli',
    'depth': 6,
    'rsm': 0.1,
    'iterations': 5000,
    'l2_leaf_reg': 37.05327214190692,
    'min_data_in_leaf': 600,
    'random_strength': 27.64341860474617,
    'subsample': 0.9,
    # early stopping
    'early_stopping_rounds':5000,
    'use_best_model': False,
}

In [10]:
def train_models(dataframe: pd.DataFrame, n_folds: int = 5,) -> tuple:
    
    models = list()
    
    # dataframe to store the oof predictions
    oof = dataframe[["target"]].copy()
    oof["pred"] = -1

    for fold in range(n_folds):
        
        print(f" training model {fold+1}/{n_folds} ".center(100, "#"))
        
        train_df = dataframe.query("fold != @fold").copy()
        valid_df = dataframe.query("fold == @fold").copy()
                
        train_dset = catboost.Pool(
            data=train_df.loc[:,input_feats],
            label=train_df.loc[:,"target"].values,
        )
        valid_dset = catboost.Pool(
            data=valid_df.loc[:,input_feats],
            label=valid_df.loc[:,"target"].values,
        )
        
        model = catboost.CatBoostClassifier(**model_params)
        model.fit(
            train_dset,
            eval_set=valid_dset,
            verbose=25,
        )
        
        #lgb.plot_importance(model, figsize=(8,15), importance_type="split", max_num_features=30)
        #lgb.plot_importance(model, figsize=(8,15), importance_type="gain", max_num_features=30)
        #plt.show()        
        
        oof.loc[valid_df.index,"pred"] = model.predict(valid_dset, prediction_type="Probability")[:,1]
        
        models.append(model)
        del train_df,valid_df,train_dset,valid_dset
        gc.collect()
    
    return models,oof

In [11]:
# implement repeated cross validation
sorted(glob("../data/processed/cv*.csv"))

['../data/processed/cv0.csv',
 '../data/processed/cv1.csv',
 '../data/processed/cv2.csv',
 '../data/processed/cv3.csv',
 '../data/processed/cv4.csv',
 '../data/processed/cv5.csv',
 '../data/processed/cv6.csv',
 '../data/processed/cv7.csv',
 '../data/processed/cv8.csv',
 '../data/processed/cv9.csv']

In [12]:
%%time 

all_models = list()
all_oof_dfs = list()

for repetition in range(N_REPEATS):
    print(f" repeated cross-validation step: {repetition+1}/{N_REPEATS} ".center(100, "#"))

    folds = pd.read_csv(f'../data/processed/cv{repetition}.csv', index_col="customer_ID")
    _train = pd.merge(train, folds, how="inner", left_index=True, right_index=True).reset_index(drop=True)
    
    tic = time.time()
    models,oof = train_models(_train)
    tac = time.time()
    print(f"Training time: {(tac-tic)/60} min.")
          
    # oof metrics
    print("OOF recall_at4:", compute_recall_at4(oof.target.values, oof.pred.values))
    print("OOF normalized_gini:", compute_normalized_gini(oof.target.values, oof.pred.values))
    print("OOF competition metric:", compute_amex_metric(oof.target.values, oof.pred.values))
    
    all_models.append(models)
    all_oof_dfs.append(oof)
    
    # save oof predictions
    oof.to_csv(OOF_PATH/f"oof-cv{repetition}.csv", index=False)
    # save models
    for fold,_model in enumerate(models):
        _model.save_model(ART_PATH/f"/model-cv{repetition}-fold{fold}.cbm", format="cbm")
          
    del _train, folds; gc.collect()

############################### repeated cross-validation step: 1/3 ################################


######################################## training model 1/5 ########################################


0:	learn: 0.5866766	test: 0.5878192	best: 0.5878192 (0)	total: 3.09s	remaining: 4h 17m 50s


25:	learn: 0.7234873	test: 0.7281298	best: 0.7281298 (25)	total: 9.52s	remaining: 30m 20s


50:	learn: 0.7418457	test: 0.7481086	best: 0.7481086 (50)	total: 16s	remaining: 25m 51s


...

4950:	learn: 0.8907844	test: 0.7955398	best: 0.7958735 (4865)	total: 21m 23s	remaining: 12.7s


4975:	learn: 0.8910984	test: 0.7956704	best: 0.7958735 (4865)	total: 21m 30s	remaining: 6.22s


4999:	learn: 0.8914557	test: 0.7956718	best: 0.7958735 (4865)	total: 21m 36s	remaining: 0us

bestTest = 0.7958734644
bestIteration = 4865



Training time: 108.80539236068725 min.
OOF recall_at4: 0.6685797960076749
OOF normalized_gini: 0.9250443910541197
OOF competition metric: 0.7968120935308973


CPU times: user 1d 7h 32min 53s, sys: 54min 26s, total: 1d 8h 27min 20s
Wall time: 5h 27min 8s


In [13]:
results = list()

for oof in all_oof_dfs:  
    r = {
        "recall_at4": compute_recall_at4(oof.target.values, oof.pred.values),
        "gini": compute_normalized_gini(oof.target.values, oof.pred.values),
        "metric": compute_amex_metric(oof.target.values, oof.pred.values),
    }
    results.append(r)

results = pd.DataFrame(results)
display(results)

print("\nmean:")
display(results.mean(axis=0))

print("\nstd:")
display(results.std(axis=0))

,recall_at4,gini,metric
0,0.668479,0.924912,0.796696
1,0.668058,0.924928,0.796493
2,0.668580,0.925044,0.796812



mean:


recall_at4    0.668372
gini          0.924961
metric        0.796667
dtype: float64


std:


recall_at4    0.000277
gini          0.000072
metric        0.000162
dtype: float64

In [14]:
del train
gc.collect()

21

***
## make predictions and submit

In [15]:
def make_predictions(dataframe:pd.DataFrame, input_feats:list, models:list) -> np.array:
    preds = [
        model.predict(dataframe[input_feats], prediction_type="Probability")[:,1] 
        for model in models
    ]
    return np.mean(preds, axis=0)    

In [16]:
test = pd.read_parquet("../data/processed/dsv06/test.parquet")
sub = pd.read_csv("../data/raw/sample_submission.csv")

In [17]:
%%time

all_preds = list()

for repetition in range(N_REPEATS):
    if "prediction" in sub.columns:
        sub.drop("prediction", axis=1, inplace=True)
    if "prediction" in test.columns:
        test.drop("prediction", axis=1, inplace=True)
        
    models = all_models[repetition]
    preds = make_predictions(test, input_feats, models)
    all_preds.append(preds)
       
    test["prediction"] = preds
    sub["prediction"] = test.loc[sub.customer_ID.values,"prediction"].values
    assert sub.prediction.isna().sum() == 0
    sub.to_csv(SUB_PATH/f"submission-cv{repetition}.csv", index=False)

CPU times: user 7min 28s, sys: 13.4 s, total: 7min 42s
Wall time: 1min 27s


In [18]:
%%time
# predict using all the trained models
if "prediction" in sub.columns:
    sub.drop("prediction", axis=1, inplace=True)
if "prediction" in test.columns:
    test.drop("prediction", axis=1, inplace=True)

test["prediction"] = np.mean(all_preds, axis=0)
sub["prediction"] = test.loc[sub.customer_ID.values,"prediction"].values
assert sub.prediction.isna().sum() == 0
sub.to_csv(SUB_PATH/f"submission-all.csv", index=False)

CPU times: user 3.6 s, sys: 810 ms, total: 4.41 s
Wall time: 4.4 s


***